In [6]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, Conv2DTranspose
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
import matplotlib.pyplot as plt

In [7]:
!pip install tensorflow-gpu

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for tensorflow-gpu
Failed to build tensorflow-gpu
  Running setup.py install for tensorflow-gpu: started
  Running setup.py install for tensorflow-gpu: finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [18 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\ACER\AppData\Local\Temp\pip-install-44v9i65b\tensorflow-gpu_5669506b82784fa8b1bc74daa8414469\setup.py", line 37, in <module>
      raise Exception(TF_REMOVAL_WARNING)
  Exception:
  
  The "tensorflow-gpu" package has been removed!
  
  Please install "tensorflow" instead.
  
  Other than the name, the two packages have been identical
  since TensorFlow 2.1, or roughly since Sep 2019. For more
  information, see: pypi.org/project/tensorflow-gpu
  
  
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
  error: subprocess-exited-with-error
  
  Running setup.py install for tensorflow-gpu did not 

# UNET++ Basic Model

In [9]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            img = cv2.imread(img_path)
            if img is not None:
                images.append(img)
    return images

In [10]:
os.getcwd()

'C:\\Users\\ACER\\Downloads\\Segmentation\\Codes'

In [11]:
# Path to the folders containing before and after segmentation images
before_seg_folder = '../dataset/Augmented Dataset Skull Stripping 3T/Augmented_before_3T'
after_seg_folder = '../dataset/Augmented Dataset Skull Stripping 3T/Augmented_after_3T'

In [12]:
# Load images from the folders
before_images = load_images_from_folder(before_seg_folder)
after_images = load_images_from_folder(after_seg_folder)

In [13]:
# Check if the number of images loaded is the same
if len(before_images) != len(after_images):
    print("Number of images in before and after segmentation folders don't match.")
else:
    print("Number of images loaded from both folders:", len(before_images))

Number of images loaded from both folders: 2448


In [14]:
def unet_plusplus(input_size=(256, 256, 3), num_classes=3):
    inputs = Input(input_size)
    
    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    # Add more convolutional and pooling layers for encoding...
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Decoder
    # Add convolutional and upsampling layers for decoding...
    up3 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv2)
    merge3 = concatenate([conv1, up3], axis=3)
    conv3 = Conv2D(64, 3, activation='relu', padding='same')(merge3)
    conv3 = Conv2D(64, 3, activation='relu', padding='same')(conv3)

    # Output layer
    outputs = Conv2D(num_classes, 1, activation='softmax')(conv3)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [15]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(before_images, after_images, test_size=0.2, random_state=42)

In [16]:
# Convert lists to numpy arrays
X_train = np.array(X_train)
X_val = np.array(X_val)
y_train = np.array(y_train)
y_val = np.array(y_val)

In [17]:
# Normalize the images to [0, 1]
X_train = X_train / 255.0
X_val = X_val / 255.0
y_train = y_train / 255.0
y_val = y_val / 255.0

In [18]:
# Define the UNet++ model
model = unet_plusplus()

In [19]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
# Train the model
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1275s 20s/step - accuracy: 0.3062 - loss: 0.3899 - val_accuracy: 0.3768 - val_loss: 0.2080
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1263s 20s/step - accuracy: 0.3665 - loss: 0.2077 - val_accuracy: 0.6405 - val_loss: 0.1971
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1222s 20s/step - accuracy: 0.4233 - loss: 0.1931 - val_accuracy: 0.1581 - val_loss: 0.1923
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1199s 19s/step - accuracy: 0.3122 - loss: 0.1916 - val_accuracy: 0.1795 - val_loss: 0.1901
Epoch 5/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1162s 19s/step - accuracy: 0.2643 - loss: 0.1886 - val_accuracy: 0.1735 - val_loss: 0.1904
Epoch 6/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1182s 19s/step - accuracy: 0.2881 - loss: 0.1885 - val_accuracy: 0.3469 - val_loss: 0.1883
Epoch 7/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1196s 19s/step - accuracy: 0.4072 - loss: 0.1902 - val_accuracy: 0.2569 - val_loss: 0.1884
Epoch 8/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1194s 19s/step - accuracy: 0.3013 - loss: 0.1873 - val_accu

In [21]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

16/16 ━━━━━━━━━━━━━━━━━━━━ 43s 3s/step - accuracy: 0.1389 - loss: 0.1871
Validation Loss: 0.18535208702087402
Validation Accuracy: 0.1385546624660492


# Updated UNET++ model

In [23]:
# Path to the folders containing before and after segmentation images
before_seg_folder = '../dataset/Augmented Dataset Skull Stripping 3T/Augmented_before_3T'
after_seg_folder = '../dataset/Augmented Dataset Skull Stripping 3T/Augmented_after_3T'

In [24]:
# Function to load and preprocess dataset
def load_dataset(mri_dir, manual_crop_dir, img_size=(256, 256)):
    # Lists to store MRI and manual crop images
    mri_images = []
    manual_crop_images = []
    
    # Load MRI images and corresponding manual crop images
    for filename in os.listdir(mri_dir):
        if filename.endswith('.png'):
            mri_path = os.path.join(mri_dir, filename)
            manual_crop_path = os.path.join(manual_crop_dir, filename)
            
            # Read and preprocess MRI image
            mri_img = cv2.imread(mri_path, cv2.IMREAD_GRAYSCALE)
            mri_img = cv2.resize(mri_img, img_size)
            mri_img = mri_img / 255.0  # Normalize to [0, 1]
            mri_images.append(mri_img)
            
            # Read and preprocess manual crop image
            manual_crop_img = cv2.imread(manual_crop_path, cv2.IMREAD_GRAYSCALE)
            manual_crop_img = cv2.resize(manual_crop_img, img_size)
            manual_crop_img = manual_crop_img / 255.0  # Normalize to [0, 1]
            manual_crop_images.append(manual_crop_img)
    
    # Convert lists to numpy arrays
    mri_images = np.array(mri_images)
    manual_crop_images = np.array(manual_crop_images)
    
    return mri_images, manual_crop_images

In [25]:
# Load dataset
before_images, after_images = load_dataset(before_seg_folder, after_seg_folder)

In [26]:
len(after_images)

2448

In [27]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(before_images, after_images, test_size=0.2, random_state=42)

In [28]:
len(X_train)

1958

In [29]:
# Define UNet++ model architecture
def unet_plusplus_modified(input_shape):
    inputs = Input(input_shape)
    
    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    # Bottleneck
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Decoder
    up6 = Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(drop5)
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(conv6)
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv7)
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv8)
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)
    return model

In [53]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [58]:
# Get the current working directory
current_directory = os.getcwd()

# Define the directory where you want to save the checkpoint
checkpoint_directory = os.path.join(current_directory, "Checkpoint")

# Ensure that the directory exists, create it if it doesn't
os.makedirs(checkpoint_directory, exist_ok=True)

# Define the checkpoint filepath
checkpoint_filepath = os.path.join(checkpoint_directory, "unetplusplus_checkpoint.keras")

In [59]:
# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,  # Save the entire model
    monitor='val_loss',  # Monitor validation loss
    mode='min',  # Minimize validation loss
    save_best_only=True  # Save only the best model
)

In [56]:
# Compile the model
input_shape = (256, 256, 1)
model = unet_plusplus_modified(input_shape)
model.compile(optimizer=Adam(), loss=BinaryCrossentropy(), metrics=['accuracy'])

In [57]:
try:
    # Train the model with checkpoint callback
    history = model.fit(
        X_train, y_train,
        batch_size=32,
        epochs=10,
        validation_data=(X_val, y_val),
        callbacks=[checkpoint_callback]
    )

    print("Training completed successfully.")

except Exception as e:
    print("An error occurred during training:", e)

finally:
    # Check if the checkpoint file was created
    if os.path.exists(checkpoint_filepath):
        print("Checkpoint file saved at:", checkpoint_filepath)
    else:
        print("Checkpoint file was not saved.")

Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 3180s 51s/step - accuracy: 0.6773 - loss: 0.4749 - val_accuracy: 0.7155 - val_loss: 0.2077
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 3159s 51s/step - accuracy: 0.7134 - loss: 0.2027 - val_accuracy: 0.7153 - val_loss: 0.1867
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 3167s 51s/step - accuracy: 0.7118 - loss: 0.1892 - val_accuracy: 0.7155 - val_loss: 0.1834
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 3179s 51s/step - accuracy: 0.7142 - loss: 0.1938 - val_accuracy: 0.7155 - val_loss: 0.1848
Epoch 5/10
18/62 ━━━━━━━━━━━━━━━━━━━━ 36:21 50s/step - accuracy: 0.7093 - loss: 0.1893Checkpoint file was not saved.


KeyboardInterrupt: 

In [ ]:
# # At this point, the model has been trained and checkpoints have been saved

# # If training is interrupted or you want to resume training from the last checkpoint:
# # Load the model from the last checkpoint
# model = keras.models.load_model(checkpoint_filepath)

# # Resume training
# history = model.fit(
#     X_train, y_train,
#     batch_size=32,
#     epochs=10,
#     validation_data=(X_val, y_val),
#     callbacks=[checkpoint_callback]
# )

In [60]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

16/16 ━━━━━━━━━━━━━━━━━━━━ 145s 9s/step - accuracy: 0.7128 - loss: 0.1861
Validation Loss: 0.18426823616027832
Validation Accuracy: 0.7154667973518372


# Training for both 3T and 7T

In [61]:
# Path to the folders containing before and after segmentation images
before_seg_folder = '../dataset/Augmented Dataset Skull Stripping 3T/Augmented_before_3T'
after_seg_folder = '../dataset/Augmented Dataset Skull Stripping 3T/Augmented_after_3T'

In [62]:
# Load dataset
before_images, after_images = load_dataset(before_seg_folder, after_seg_folder)

In [63]:
len(before_images)

2448

In [64]:
# Path to the folders containing before and after segmentation images
before_seg_folder = '../dataset/Augmented Dataset Skull Stripping 7T/Augmented_before_7T'
after_seg_folder = '../dataset/Augmented Dataset Skull Stripping 7T/Augmented_after_7T'

In [65]:
# Load dataset
before_images_7T, after_images_7T = load_dataset(before_seg_folder, after_seg_folder)

In [66]:
len(before_images_7T)

2532

In [71]:
before_images = np.concatenate((before_images, before_images_7T), axis=0)

In [72]:
len(before_images)

4980

In [73]:
after_images = np.concatenate((after_images, after_images_7T), axis=0)

In [74]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(before_images, after_images, test_size=0.2, random_state=42)

In [75]:
len(X_train)

3984

In [76]:
# Get the current working directory
current_directory = os.getcwd()

# Define the directory where you want to save the checkpoint
checkpoint_directory = os.path.join(current_directory, "Checkpoint")

# Ensure that the directory exists, create it if it doesn't
os.makedirs(checkpoint_directory, exist_ok=True)

# Define the checkpoint filepath
checkpoint_filepath = os.path.join(checkpoint_directory, "unetplusplus_both_checkpoint.keras")

In [77]:
# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,  # Save the entire model
    monitor='val_loss',  # Monitor validation loss
    mode='min',  # Minimize validation loss
    save_best_only=True  # Save only the best model
)

In [78]:
# Compile the model
input_shape = (256, 256, 1)
model = unet_plusplus_modified(input_shape)
model.compile(optimizer=Adam(), loss=BinaryCrossentropy(), metrics=['accuracy'])

In [79]:
try:
    # Train the model with checkpoint callback
    history = model.fit(
        X_train, y_train,
        batch_size=32,
        epochs=5,
        validation_data=(X_val, y_val),
        callbacks=[checkpoint_callback]
    )

    print("Training completed successfully.")

except Exception as e:
    print("An error occurred during training:", e)

finally:
    # Check if the checkpoint file was created
    if os.path.exists(checkpoint_filepath):
        print("Checkpoint file saved at:", checkpoint_filepath)
    else:
        print("Checkpoint file was not saved.")

Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 6470s 52s/step - accuracy: 0.6897 - loss: 1.3278 - val_accuracy: 0.7161 - val_loss: 1.4043
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 6402s 51s/step - accuracy: 0.7185 - loss: 1.4112 - val_accuracy: 0.7161 - val_loss: 1.4043
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 6440s 52s/step - accuracy: 0.7181 - loss: 1.4131 - val_accuracy: 0.7161 - val_loss: 1.4043
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 6434s 51s/step - accuracy: 0.7155 - loss: 1.4305 - val_accuracy: 0.7161 - val_loss: 1.4043
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 6471s 52s/step - accuracy: 0.7173 - loss: 1.4114 - val_accuracy: 0.7161 - val_loss: 1.4043
Training completed successfully.
Checkpoint file saved at: C:\Users\ACER\Downloads\Segmentation\Codes\Checkpoint\unetplusplus_both_checkpoint.keras


In [80]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

32/32 ━━━━━━━━━━━━━━━━━━━━ 293s 9s/step - accuracy: 0.7160 - loss: 1.4198
Validation Loss: 1.4042959213256836
Validation Accuracy: 0.7161155343055725
